In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_add_label_4_lr_0.1_inculde_networks.parquet')
DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_add_label_5_lr_0.03_inculde_networks.parquet')
DATA.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,avg_gpu3_core_temp_max,std_gpu3_core_temp_max,min_gpu3_core_temp_max,max_gpu3_core_temp_max,avg_ps0_input_power_min,std_ps0_input_power_min,min_ps0_input_power_min,max_ps0_input_power_min,next_value,timestamp
0,22.632605,0.082797,22.509747,22.685978,30.431459,0.138977,30.166260,30.585276,30.848211,0.136500,...,44.083305,6.958302,36.521801,53.254215,341.266022,75.682755,257.505951,442.272919,0.017987,2021-04-02 00:45:00+00:00
1,22.635746,0.083260,22.511705,22.689707,30.434170,0.138600,30.169426,30.589201,30.851128,0.136243,...,44.083694,6.961399,36.526020,53.262257,341.346680,75.735840,257.599915,442.455353,0.017967,2021-04-02 01:00:00+00:00
2,22.638121,0.084007,22.515509,22.694950,30.438974,0.139176,30.173515,30.597065,30.856789,0.135928,...,44.093174,6.962402,36.531658,53.275291,341.457275,75.766037,257.665527,442.639343,0.017918,2021-04-02 01:15:00+00:00
3,22.636614,0.083642,22.515266,22.696047,30.436201,0.138888,30.173256,30.596045,30.855408,0.135764,...,44.086926,6.957322,36.531876,53.266701,341.443573,75.814156,257.640503,442.686920,0.019866,2021-04-02 01:30:00+00:00
4,22.636734,0.083944,22.516420,22.699638,30.436991,0.139211,30.175510,30.597576,30.854156,0.135490,...,44.097366,6.955503,36.543251,53.277370,341.487854,75.797501,257.715240,442.693054,0.019795,2021-04-02 01:45:00+00:00


In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
DATA = DATA.astype(float)

In [ ]:
scaler = preprocessing.MinMaxScaler()
names = DATA.columns
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)
### parisa change
timestamp_scaler = preprocessing.MinMaxScaler()
DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
# DATA['next_value']=DATA3['next_value']*100/4
# DATA['next_value'][0]=0.44665

In [ ]:
# DATA

In [ ]:
# DATA.dropna()

In [ ]:
### to get rid of time and test
# DATA=DATA.drop(columns=['timestamp'])

In [ ]:
train_data = DATA[:int(DATA.shape[0]*0.8)]
test_data = DATA[int(DATA.shape[0]*0.8):]

In [ ]:
train_data['value'].value_counts()

,count
value,
0.449137,2
0.488393,2
0.400800,2
0.404007,2
0.317467,2
...,...
0.282988,1
0.283525,1
0.282731,1


In [ ]:
train_feat = train_data.drop(columns=['value'])
train_feat = train_feat.to_numpy()
test_feat = test_data.drop(columns=['value'])
test_feat = test_feat.to_numpy()

In [ ]:
train_feat = torch.tensor(train_feat, dtype=torch.float)
test_feat = torch.tensor(test_feat, dtype=torch.float)

In [ ]:
test_labels = test_data['value']
test_labels = test_labels.to_numpy()
test_labels = torch.tensor(test_labels, dtype=torch.float)
test_labels

tensor([0.4162, 0.4165, 0.4171,  ..., 0.3144, 0.3110, 0.3088])

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq


In [ ]:
# train_seq = create_data_seq(train_feat,train_window,revelent_len) ## parisa change
train_seq = create_data_seq(train_feat,train_window) ## parisa change

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader))
# print(X)


In [ ]:
# test_seq = create_data_seq(test_feat,train_window,revelent_len)
test_seq = create_data_seq(test_feat,train_window)

In [ ]:
test_loader = DataLoader(test_seq, batch_size=1, shuffle=False) ## در هر فور ما یه تعدادی بچ داریم که میدیم به دل با بچ بیشتر میتونیم بگیم مثلا 2 تا 2 تا بهش بده
X,y = next(iter(test_loader))

In [ ]:
############### mine for add h,c

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=1382 ,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=1382,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 1382)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 1382)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(1382, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 1382, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=1382, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=1

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(1382, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 1382, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=1382, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=1

In [ ]:
%%time
def train():
    model.train()
    for d in train_loader:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss)

for epoch in range(6):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')




############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351
# Epoch: 01, Loss: 0.0111
# Epoch: 02, Loss: 0.2479
# Epoch: 03, Loss: 0.0097
# Epoch: 04, Loss: 0.0052
# Epoch: 05, Loss: 0.0080
# Epoch: 06, Loss: 0.0088
# CPU times: user 41min 57s, sys: 15.8 s, total: 42min 12s
# Wall time: 42min 38s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351
# Epoch: 01, Loss: 0.0080
# Epoch: 02, Loss: 0.0062
# Epoch: 03, Loss: 0.0100
# Epoch: 04, Loss: 0.2012
# Epoch: 05, Loss: 0.0097
# Epoch: 06, Loss: 0.0084
# CPU times: user 47min 20s, sys: 21.4 s, total: 47min 41s
# Wall time: 48min 17s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6  ,len(data)=1351
# Epoch: 01, Loss: 0.0100
# Epoch: 02, Loss: 0.0079
# Epoch: 03, Loss: 0.0118
# Epoch: 04, Loss: 0.0127
# Epoch: 05, Loss: 0.2008
# Epoch: 06, Loss: 0.1996
# CPU times: user 1h 1min 51s, sys: 48.3 s, total: 1h 2min 40s
# Wall time: 1h 1min 51s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1350 (remove time)
# Epoch: 01, Loss: 0.0132
# Epoch: 02, Loss: 0.0073
# Epoch: 03, Loss: 0.0063
# Epoch: 04, Loss: 0.0073
# Epoch: 05, Loss: 0.0050
# Epoch: 06, Loss: 0.0042
# CPU times: user 40min 43s, sys: 12.6 s, total: 40min 56s
# Wall time: 41min 23s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time)
# Epoch: 01, Loss: 0.0123
# Epoch: 02, Loss: 0.0086
# Epoch: 03, Loss: 0.0139
# Epoch: 04, Loss: 0.0281
# Epoch: 05, Loss: 0.0211
# Epoch: 06, Loss: 0.0112
# CPU times: user 41min 51s, sys: 11.9 s, total: 42min 3s
# Wall time: 42min 32s


########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value
# Epoch: 01, Loss: 0.0104
# Epoch: 02, Loss: 0.0312
# Epoch: 03, Loss: 0.0147
# Epoch: 04, Loss: 0.0170
# Epoch: 05, Loss: 0.0135
# Epoch: 06, Loss: 0.0116
# CPU times: user 41min 27s, sys: 10.2 s, total: 41min 37s
# Wall time: 42min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value , lr=0.03 in data
# Epoch: 01, Loss: 0.0389
# Epoch: 02, Loss: 0.0300
# Epoch: 03, Loss: 0.0065
# Epoch: 04, Loss: 0.0226
# Epoch: 05, Loss: 0.0122
# Epoch: 06, Loss: 0.0110
# CPU times: user 42min 44s, sys: 13.1 s, total: 42min 57s
# Wall time: 41min 44s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value , lr=0.03 in data and lr 0.05 in detection
# Epoch: 01, Loss: 0.0129
# Epoch: 02, Loss: 0.0109
# Epoch: 03, Loss: 0.0065
# Epoch: 04, Loss: 0.0189
# Epoch: 05, Loss: 0.0124
# Epoch: 06, Loss: 0.0080
# CPU times: user 43min 24s, sys: 13.1 s, total: 42min 57s
# Wall time: 41min 34s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value , lr=0.03 in data and lr 0.07 in detection
# Epoch: 01, Loss: 0.0117
# Epoch: 02, Loss: 0.0056
# Epoch: 03, Loss: 0.0075
# Epoch: 04, Loss: 0.0077
# Epoch: 05, Loss: 0.0088
# Epoch: 06, Loss: 0.0110
# CPU times: user 42min 49s, sys: 12.9 s, total: 43min 2s
# Wall time: 43min 44s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value , lr=0.03 in data and lr 0.09 in detection
# Epoch: 01, Loss: 0.0097
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0081
# Epoch: 04, Loss: 0.0130
# Epoch: 05, Loss: 0.0092
# Epoch: 06, Loss: 0.0059
# CPU times: user 42min 23s, sys: 13 s, total: 42min 36s
# Wall time: 43min 6s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382 (with time) change in next_value , lr=0.03 in data and lr 0.3 in detection
# Epoch: 01, Loss: 0.0065
# Epoch: 02, Loss: 0.0101
# Epoch: 03, Loss: 0.0100
# Epoch: 04, Loss: 0.0058
# Epoch: 05, Loss: 0.0058
# Epoch: 06, Loss: 0.0031
# CPU times: user 42min 32s, sys: 12.4 s, total: 42min 44s
# Wall time: 43min 13s

In [ ]:
def test(x):
    model.eval()
    x = x.to(device)
    out = model(x)
    return out

In [ ]:
from sklearn.metrics import mean_squared_error,roc_auc_score
reconstruction = []
loss = []
#y_true = []
for d in test_loader:
    x = d[0][0]
    y = d[1]
    out = test(x)
    reconstruction.append(out)
    loss.append(mean_squared_error(out[0][train_window-1].detach().cpu().numpy(),y[0][0]))
    #y_true.append(d[1].detach().cpu().numpy())

In [ ]:
# test_seq_label = create_data_seq(test_labels,train_window,revelent_len)
test_seq_label = create_data_seq(test_labels,train_window)

In [ ]:
temp_list = []
for i in range(len(test_seq_label)):
    temp = test_seq_label[i][1].type(torch.int64)
    temp = temp.detach().cpu().numpy().tolist()
    temp_list.append(temp)

In [ ]:
y_true = []
for i in range(len(temp_list)):
    y_true.append(temp_list[i][0])

In [ ]:
error_df = pd.DataFrame({'loss': loss,
                        'true_class': y_true})
error_df.describe()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(error_df['true_class'],error_df['loss'])

In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
#0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c --- moshtrak
#0.9345063164717007    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351, with time
#0.6694775694129658    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351 , with time
#0.6529441138631129    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 ,len(data)=1350, with time
#0.9342669032465992    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1350, (remove time)
#0.28216920571045323    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382, (with time) , lr =0.03
#0.7664079716393599    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382, (with time) , lr =0.03 for model to detect lr=0.05
#0.7617131359586088    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382, (with time) , lr =0.03 for model to detect lr=0.07
#0.7964932451853981    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382, (with time) , lr =0.03 for model to detect lr=0.09
#0.8641372041774456    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1382, (with time) , lr =0.03 for model to detect lr=0.3